In [ ]:
import os
import warnings
from typing import Literal
from typing_extensions import TypedDict

from dotenv import load_dotenv
from langchain.agents import load_tools
from langchain_anthropic import ChatAnthropic
from langchain_community.tools import ShellTool
from langchain_community.tools import BraveSearch
from langchain_community.agent_toolkits import FileManagementToolkit
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.prebuilt import create_react_agent
from langgraph.types import Command

load_dotenv()
warnings.filterwarnings('ignore')

In [ ]:
llm = ChatAnthropic(model="claude-3-7-sonnet-20250219", max_retries=5, max_tokens=16384, temperature=0)

bash_tool = ShellTool()
search_tool = BraveSearch.from_api_key(api_key=os.getenv("BRAVE_API_KEY"), search_kwargs={"count": 3})
fs_tool = FileManagementToolkit(root_dir="./").get_tools()
human = load_tools(
    ["human"],
    llm=llm,
)

### Agent Supervisor

It will use LLM with structured output to choose the next worker node OR finish processing.

In [ ]:
team_members = ["frontend developer", "backend developer", "devops engineer"]
# Our team supervisor is an LLM node. It just picks the next agent to process and decides when the work is completed
options = team_members + ["FINISH"]

system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    f" following workers: {team_members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)

available_tools = [bash_tool, search_tool] + fs_tool + human

class Router(TypedDict):
    """Worker to route to next. If no workers needed, route to FINISH."""

    next: Literal[*options]


class State(MessagesState):
    next: str


def supervisor_node(state: State) -> Command[Literal[*team_members, "__end__"]]:
    messages = [
        {"role": "system", "content": system_prompt},
    ] + state["messages"]
    response = llm.with_structured_output(Router).invoke(messages)
    goto = response["next"]
    if goto == "FINISH":
        goto = END

    return Command(goto=goto, update={"next": goto})

## Apes together strong 🦍🦍🦍

In [ ]:
frontend_agent = create_react_agent(
    llm,
    tools=available_tools,
    prompt=(
        "You are a frontend developer. Create frontend applications in framework "
        "requested by user. Frontend should work together with backend created by"
        " backend developer. You can ask backend or devops engineer for help, you can also ask for clarifications from the human client"
    )
)


async def frontend_node(state: State) -> Command[Literal["supervisor"]]:
    result = await frontend_agent.ainvoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="frontend developer")
            ]
        },
        goto="supervisor",
    )


backend_agent = create_react_agent(
    llm,
    tools=available_tools,
    prompt=(
        "You are a backend developer. Create backend part of the applications in framework "
        "requested by user. Backend should work together with frontend created by frontend "
        "developer. You can ask frontend or devops engineer for help, you can also ask for clarifications from the human client"
    )
)


async def backend_node(state: State) -> Command[Literal["supervisor"]]:
    result = await backend_agent.ainvoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="backend developer")
            ]
        },
        goto="supervisor",
    )


devops_agent = create_react_agent(
    llm,
    tools=available_tools,
    prompt=(
        "You are a devops engineer. Help backend and frontend engineers to deploy their "
        "applications as instructed in the user request. You can ask frontend or backend "
        "developer any clarification questions about their code, you can also ask for clarifications from the human client."
    )
)

async def devops_node(state: State) -> Command[Literal["supervisor"]]:
    result = await devops_agent.ainvoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="devops engineer")
            ]
        },
        goto="supervisor",
    )


builder = StateGraph(State)
builder.add_edge(START, "supervisor")
builder.add_node("supervisor", supervisor_node)
builder.add_node("frontend developer", frontend_node)
builder.add_node("backend developer", backend_node)
builder.add_node("devops engineer", devops_node)
graph = builder.compile()

In [ ]:
from IPython.display import display, Image

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except:
    graph.get_graph().print_ascii()

## Run the task

In [ ]:
def formatting(s):
    node, event = s
    if len(node) == 0:
        print("Entering graph")
        print(event)
        return
    agent_type = node[0].split(':')[0]
    print(f"\n\033[92mCurrent agent\033[0m - {agent_type}")
    event_type = list(event.keys())[0]
    if event_type == "tools":
        if event[event_type]['messages'][0].content:
            print(f"\033[94mTool call result\033[0m: {event[event_type]['messages'][0].content}")
    elif event_type == "agent":
        content = event[event_type]['messages'][0].content
        if isinstance(content, str):
            print(f"\033[92m{agent_type}\033[0m: {content}")
            return
        agent_messages = list(filter(lambda x: x["type"] == "text", content))
        if agent_messages:
            print(f"\033[92m{agent_type}\033[0m: {agent_messages[0]['text']}")
        tools = list(filter(lambda x: x["type"] == "tool_use", content))
        if tools:
            for tool in tools:
                if tool["input"]:
                    print(f"\033[92m{agent_type}\033[0m: calling tool \033[93m{tool['name']} \033[0mwith the following input:")
                    for key, value in tool["input"].items():
                        print(f"\033[96m{key}\033[0m: \033[97m{value}\033[0m")
                else:
                    print(f"\033[92m{agent_type}\033[0m: using tool \033[93m{tool['name']}\033[0m")
    else:
        print("event", event)

In [ ]:
user_prompt = (
    "I want to build a website for a conference, it should have several pages, "
    "namely: 1. Intro page about conference, 2. Page for people to submit their talks, "
    "3. Page with submitted talks. Frontend part needs to be written in react, backend - in fastapi. "
    "I want to store the submissions in postgresql database. "
    "In the end run the project in docker and docker compose and give me the local url to test. "
    "You can ask human client for any clarifications"
)
async for s in graph.astream(
    {"messages": [("user", user_prompt)]},
    {"recursion_limit": 100},
    subgraphs=True,
    stream_mode="updates",
):
    formatting(s)
    print("-" * 30)